In [1]:
!pip install optuna
!pip install optuna_integration
!pip install scikeras
!pip install -U cupy-cuda12x

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 MB 7.6 MB/s eta 0:00:00
  Attempting uninstall: cupy-cuda12x
    Found existing installation: cupy-cuda12x 13.3.0
    Uninstalling cupy-cuda12x-13.3.0:
      Successfully uninstalled cupy-cuda12x-13.3.0


In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import copy
import os
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.pruners import HyperbandPruner
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split,StratifiedKFold, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import mutual_info_regression
from cuml.preprocessing import TargetEncoder

df_train = pd.read_csv("/content/drive/MyDrive/train.csv",index_col='id') #
df_train_ex=pd.read_csv("/content/drive/MyDrive/training_extra.csv",index_col='id')
df_test = pd.read_csv("/content/drive/MyDrive/test.csv",index_col='id')
df_train=pd.concat([df_train, df_train_ex], axis=0,ignore_index=True) #Data integration/merging of the training dataset with the extended dataset.
df_train=df_train.reset_index(drop=True)
# df_train.index = df_train['id']
# df_train.index.name = None
#df_train.rename(columns={df_train.columns[0]: ""}, inplace=True)
print(df_train)
#df_train.reset_index(drop=True, inplace=True)  # Reset the index
#df_train.set_index("id", inplace=True)
#df_train["id"] = df_train.index
X_train = df_train.drop(columns=['Price']) #We only want the features in X_train, so we remove Price (the target variable). We also remove 'id' as it does not influence the price of the backpack. 'Id' is only an identifier for the sample.
y_train = df_train['Price']
print(X_train)
print(y_train)
print(X_train.index)
X_test= copy.deepcopy(df_test) #This creates a duplicate copy of df_test. Note that since this is a deepcopy, any changes to X_test will not affect df_test, as opposed to a shallow copy where both objects will be changed.
#X_test.drop(columns=['id'],inplace=True)

print(X_train.dtypes)
print(X_train.isnull().sum()) #Checks that number of missing values in each column
print(X_test.isnull().sum())

#X_train['Compartments'].fillna(X_train['Compartments'].median(), inplace=True) #Replacing missing values in numeric columns with the median of that column, for the training data.
X_train['Weight Capacity (kg)'].fillna(X_train['Weight Capacity (kg)'].median(), inplace=True)

#X_test['Compartments'].fillna(X_test['Compartments'].median(), inplace=True) #Missing value imputation for testing data, indentical process to training data.
X_test['Weight Capacity (kg)'].fillna(X_train['Weight Capacity (kg)'].median(), inplace=True)
# def impute_categorical_uniform(df, columns):
#     """
#     Imputes missing values in a categorical column using a uniform distribution.

#     Args:
#     df (pd.DataFrame): Input dataframe.
#     columns: List of column names to impute.

#     Returns:
#     pd.DataFrame: Dataframe with imputed values.
#     """
#     df_copy = df.copy()
#     for column in columns:
#         # Find unique categories (excluding NaN)

#         # Find unique categories (excluding NaN)
#         #unique_categories = df[column].dropna().unique()

#     # Count missing values
#         num_missing = df[column].isna().sum()

#     # If there are missing values, sample from the unique categories
#         if num_missing > 0:


#                 # Sample based on observed category probabilities
#             category_probs = df_copy[column].value_counts(normalize=True)
#             imputed_values = np.random.choice(category_probs.index, size=num_missing, p=category_probs.values)
#             df_copy.loc[df_copy[column].isna(), column] = imputed_values

#     return df_copy




# X_train=impute_categorical_uniform(X_train, columns=['Brand', 'Material', 'Laptop Compartment', 'Waterproof', 'Style', 'Color', 'Size'])
# X_test = impute_categorical_uniform (X_test, columns=['Brand', 'Material', 'Laptop Compartment', 'Waterproof', 'Style', 'Color', 'Size'])

obj_cols = X_test.select_dtypes(include=['object']).columns
X_train[obj_cols] = X_train[obj_cols].fillna('None')
X_test[obj_cols] = X_test[obj_cols].fillna('None')

# def target_encode_blended(X_train, y_train, X_test, cat_columns, alpha=10, n_splits=25):  #target encoding with smoothing parameter alpha and cross validation. This converts the categorical columns to numeric, using the corresponding mean values of the dependent variable
#     """
#     Performs target encoding with blending without concatenating X_train and y_train.

#     Args:
#         X_train (pd.DataFrame): Training feature dataframe (without the target column).
#         y_train (pd.Series): Target variable.
#         cat_columns (list): List of categorical columns to encode.
#         alpha (int): Smoothing factor (higher values make it closer to the global mean).
#         n_splits (int): Number of splits for KFold cross-validation.

#     Returns:
#         pd.DataFrame: Transformed training dataset with encoded columns.
#     """
#     train_encoded = X_train.copy()
#     test_encoded = X_test.copy()
#     global_mean = y_train.mean()

#     # Initialize new encoded columns
#     for col in cat_columns:
#         train_encoded[col + '_target_enc'] = np.nan

#     # KFold cross-validation for encoding
#     kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

#     for train_idx, val_idx in kf.split(X_train):
#         fold_X_train, fold_X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
#         fold_y_train = y_train.iloc[train_idx]

#         for col in cat_columns:
#             # Compute smoothed mean for each category
#             category_means = fold_y_train.groupby(fold_X_train[col]).agg(['mean', 'count'])
#             category_means['smooth'] = (category_means['count'] * category_means['mean'] + alpha * global_mean) / (category_means['count'] + alpha)

#             # Map smoothed values to the validation set
#             train_encoded.loc[X_train.index[val_idx], col + '_target_enc'] = fold_X_val[col].map(category_means['smooth'])

#     final_mapping = {}
#     for col in cat_columns:
#         category_means = y_train.groupby(X_train[col]).agg(['mean', 'count'])
#         category_means['smooth'] = (category_means['count'] * category_means['mean'] + alpha * global_mean) / (category_means['count'] + alpha)

#         final_mapping[col] = category_means['smooth']

#         test_encoded[col + '_target_enc'] = X_test[col].map(final_mapping[col])


#     return train_encoded,test_encoded

TE = TargetEncoder(n_folds=25, smooth=10, split_method='random', stat='mean')

features = X_test.columns.tolist()

for col in features:
    TE.fit(X_train[col],y_train)
    X_train[col] = TE.transform(X_train[col])
    X_test[col] = TE.transform(X_test[col])



scaler = StandardScaler()     #standardisation of features to have a mean of 0 and variance of 1.
columns_to_standardize = X_test.columns #['Compartments', 'Weight Capacity (kg)']
X_train[columns_to_standardize] = scaler.fit_transform(X_train[columns_to_standardize])
X_test[columns_to_standardize] = scaler.transform(X_test[columns_to_standardize])


                Brand   Material    Size  Compartments Laptop Compartment  \
0            Jansport    Leather  Medium           7.0                Yes   
1            Jansport     Canvas   Small          10.0                Yes   
2        Under Armour    Leather   Small           2.0                Yes   
3                Nike      Nylon   Small           8.0                Yes   
4              Adidas     Canvas  Medium           1.0                Yes   
...               ...        ...     ...           ...                ...   
3994313          Nike     Canvas     NaN           3.0                Yes   
3994314          Puma    Leather   Small          10.0                Yes   
3994315      Jansport     Canvas   Large          10.0                 No   
3994316          Puma     Canvas     NaN           2.0                 No   
3994317  Under Armour  Polyester  Medium           2.0                Yes   

        Waterproof      Style  Color  Weight Capacity (kg)      Price  
0  

<ipython-input-3-c981278e6a41>:47: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train['Weight Capacity (kg)'].fillna(X_train['Weight Capacity (kg)'].median(), inplace=True)
<ipython-input-3-c981278e6a41>:50: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing '

In [4]:

from tensorflow.keras import mixed_precision
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

#mixed_precision.set_global_policy('mixed_float16')
print(tf.test.is_built_with_cuda())  # Should return True
print(tf.test.is_gpu_available())
print("Available GPUs:", tf.config.list_physical_devices('GPU'))
tf.config.optimizer.set_jit(False) #disable XLA compilation as I was getting an error with it.


tf.keras.mixed_precision.set_global_policy('float32')
gpus = tf.config.list_physical_devices('GPU')
AUTOTUNE = tf.data.AUTOTUNE

if gpus:
    try:
        # Set TensorFlow to use only the first GPU
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU enabled:", gpus[0])
    except RuntimeError as e:
        print(e)

# #Define K for K-Fold Cross-Validation
# K = 5  # Number of folds
# kf = KFold(n_splits=K, shuffle=True, random_state=42)  # Ensure randomness

#target_encode_columns = ['Brand', 'Material', 'Laptop Compartment', 'Waterproof', 'Style', 'Size', 'Color']



# #Optuna hyperparameter tuning framework for optimisation.
# def objective(trial):

#     #Create fresh copies to avoid modifying original data
#      X_train_local = copy.deepcopy(X_train)
#      X_test_local = copy.deepcopy(X_test)
#      y_train_local = copy.deepcopy(y_train)

#      alpha = trial.suggest_float("alpha", 10, 12,step=1)
#      #learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
#      #l1_l2_reg = trial.suggest_loguniform("l1_l2_reg", 1e-5, 1e-2).              #here we define the possible values for the hyperparameters to test. The hyperparameters are alpha (smoothing parameter for target encoding), the learning rate, the number of neurons in each layer, regularisation term and the batch size.
#      #num_units_1 = trial.suggest_int("num_units_1", 64, 512, step=32)
#      #num_units_2 = trial.suggest_int("num_units_2", 32, 256, step=16)
#      #num_units_3= trial.suggest_int("num_units_3", 16, 128, step=16)
#      #num_units_4= trial.suggest_int("num_units_4", 8, 64, step=16)
#      #batch_size = trial.suggest_categorical("batch_size", [128, 256, 512]).
#      print(trial.params)
#      X_train_local, X_test_local =target_encode_blended(X_train_local,y_train_local, X_test_local, target_encode_columns, alpha=alpha)
#      # Drop original categorical columns (they are replaced with target encodings)
#      X_train_local.drop(columns=target_encode_columns, inplace=True)
#      X_test_local.drop(columns=target_encode_columns, inplace=True)



# #Store RMSE for each fold
#      val_rmse_scores = []

# # Loop through K-folds
#      for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_local)):  # Assuming X_train, y_train are defined
#       print(f"\n Training Fold {fold+1}/{K}...")

#       X_train_sub, X_val = X_train_local.iloc[train_idx], X_train_local.iloc[val_idx]
#       y_train_sub, y_val = y_train_local.iloc[train_idx], y_train_local.iloc[val_idx]
#       train_dataset = tf.data.Dataset.from_tensor_slices((X_train_sub, y_train_sub))
#       train_dataset = train_dataset.batch(2048).prefetch(tf.data.AUTOTUNE)
#       val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(2048).prefetch(tf.data.AUTOTUNE)



#       model = keras.Sequential([
#          keras.layers.Input(shape=(X_train_sub.shape[1],)),
#         #  keras.layers.Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.005)),
#         #  keras.layers.Dropout(0.1),
#           #keras.layers.Dense(256,activation='relu',kernel_regularizer=regularizers.l2(0.005)),
#         #  keras.layers.Dropout(0.1),
#           keras.layers.Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.0001)),
#           #keras.layers.BatchNormalization(),
#           #keras.layers.Dropout(0.2),
#          keras.layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.0001)),
#          #keras.layers.BatchNormalization(),
#          #keras.layers.Dropout(0.1),
#          keras.layers.Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.0001)),
#          #keras.layers.BatchNormalization(),
#          #keras.layers.Dropout(0.2),
#          keras.layers.Dense(1, activation='linear')
#                                                       ])  #

#       optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
#       model.compile(optimizer=optimizer, loss='mse', metrics=[keras.metrics.RootMeanSquaredError()])
#       early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) #early stopping ensure that the training process terminates if the validation loss does not decrease after a certain number of epochs. In this case 5.
#       lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6) #dynamically adjusts the learning rate, if the validation loss does not decrease after a set number of epochs.
# # pruning_callback = TFKerasPruningCallback(trial, "val_root_mean_squared_error")

#       history = model.fit(train_dataset,
#                         epochs=20,
#                         verbose=1, validation_data=val_dataset,callbacks=[lr_scheduler])

#     # Return validation RMSE as the score to minimize
#       val_rmse = history.history['val_root_mean_squared_error'][-1]  # Last epoch RMSE to be the RMSE for the particular fold.
#       val_rmse_scores.append(val_rmse)  # Store RMSE score
#       print(f"✅ Fold {fold+1} RMSE: {val_rmse:.4f}")


#       mean_rmse = np.mean(val_rmse_scores) #find average over all folds to be the RMSE for the cross validation and this is the RMSE for the particular combination of hyperparameters.
#       std_rmse = np.std(val_rmse_scores)
#       print(f"Cross-validation RMSE: {mean_rmse:.4f} ± {std_rmse:.4f}")

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=1)


# X_train, X_test =target_encode_blended(X_train,y_train, X_test, target_encode_columns, alpha=20)
# X_train.drop(columns=target_encode_columns, inplace=True) #using inplace=True ensures that we do not need to create a new dataframe with the alterations. The existing dataframe is updated with the changes.
# X_test.drop(columns=target_encode_columns, inplace=True)


X_train_id, X_val_id = train_test_split(X_train.index, test_size=0.2, random_state=42)
X_train, X_val = X_train.iloc[X_train_id], X_train.iloc[X_val_id]
y_train, y_val = y_train.iloc[X_train_id], y_train.iloc[X_val_id]
final_model = keras.Sequential([
         keras.layers.Input(shape=(X_train.shape[1],)),
#         #  keras.layers.Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.005)),
#         #  keras.layers.Dropout(0.1),
           #keras.layers.Dense(256,activation='relu'),
           #keras.layers.BatchNormalization(),
           keras.layers.Dropout(0.1),
           keras.layers.Dense(128,kernel_regularizer=regularizers.l2(0.0001)),   #128 neurones in the first layer.
           keras.layers.BatchNormalization(),
           keras.layers.Activation('relu'),
            #keras.layers.LeakyReLU(alpha=0.1), #LeakyRelu activation function, which must be defined in a separate layer, as opposed to to standard Relu, which can be defined in the same layer.
           #keras.layers.BatchNormalization(),
          keras.layers.Dropout(0.1), #dropout layer used to reduce overfitting by randomly deactivating a proportion of neurons, by setting them to 0. Noise is introduced into the training process.
          keras.layers.Dense(128,kernel_regularizer=regularizers.l2(0.0001)), #128 neurons in the second layer
            #keras.layers.LeakyReLU(alpha=0.1),


          keras.layers.BatchNormalization(), #BatchNormalisation stabilises training process.
          keras.layers.Activation('relu'),
          keras.layers.Dropout(0.1),
          keras.layers.Dense(64,kernel_regularizer=regularizers.l2(0.0001)), #64 neurons in the third layer

          #keras.layers.LeakyReLU(alpha=0.1),
          keras.layers.BatchNormalization(),
          keras.layers.Activation('relu'),
          keras.layers.Dropout(0.1),
          #keras.layers.Dropout(0.1),
          keras.layers.Dense(1, activation='linear')
                                                       ]) #one output neuron in the final layer, which represents the predicted price of tbe backpack



optimizer = tf.keras.optimizers.Adam() #Adam (Adaptive Moment estimation) optimiser is used here for stochastic gradient descent.
final_model.compile(optimizer=optimizer, loss='mse', metrics=[keras.metrics.RootMeanSquaredError()]) #loss function is mse and evaluation metric is root mean squared error, both are logged after each epoch.
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'Epoch {epoch+1}/{self.params["epochs"]} - Val RMSE: {logs["val_root_mean_squared_error"]}')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) #restore_best_weights=True ensures that tensorflow restores the most optimal parameters, that give the lowest validation loss.
#lr_scheduler = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=1e-6)
full_train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(2048).prefetch(tf.data.AUTOTUNE) #mini batch size of 2048 samples. prefetching reduces latency, as one batch is training, the next batch is loading in parallel. This is more efficient instead of the model waiting for the pipeline to load the next batch before continuing with the training process.
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(2048).prefetch(tf.data.AUTOTUNE)

history = final_model.fit(full_train_dataset, epochs=30, verbose=0,callbacks=[early_stopping,CustomCallback()],validation_data=val_dataset) #the higher the value of verbose, the more detailed logs are output.
y_pred=final_model.predict(X_test) #Predicting on the testing data.

y_pred = pd.DataFrame(y_pred, columns=["prediction"])  # Ensure predictions are converted to a dataframe and that the dataframe has a column name.

# Concatenate X_test["id"] with y_pred_df along axis=1 (column-wise)
y_pred = pd.concat([df_test['id'], y_pred], axis=1) # We add the id column back to y_pred, which is required for kaggle submission. Since X_test was a deepcopy, the id column is still present in df_test.

# Save to CSV
y_pred.to_csv("y_pred.csv", index=False)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True
True
Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU enabled: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Epoch 1/30 - Val RMSE: 38.88510513305664
Epoch 2/30 - Val RMSE: 38.674537658691406
Epoch 3/30 - Val RMSE: 38.66902160644531


KeyboardInterrupt: 